By combining the **pipeline** and **Spark** capabilities in **Fabric**, you can implement complex data ingestion logic that copies data from external sources into the **OneLake** storage on which the lakehouse is based, and then uses Spark code to perform custom data transformations before loading it into tables for analysis.

## Create Workspace ##
1) Navigate to the [Microsoft Fabric home page](https://app.fabric.microsoft.com/home?experience=fabric)
2) In the menu bar on the left, select Workspaces (the icon looks similar to 🗇)
3) Create a new workspace with a name of your choice, selecting a licensing mode that includes Fabric capacity (Trial, Premium, or Fabric).

## Create Lakehouse ##
1) Inside the workspace, click **+ New**.
2) Select Lakehouse under the **Data Engineering** section.
3) Click **Create**.

Once created, you Lakehouse will have:
- Tables folder → structured Delta tables
- Files folder → unstructured / raw data
- Built-in SQL endpoint for querying
- Auto-created Power BI semantic model

## Create a pipeline (Copy Data activity) ##
1) **Home** page (lakehouse) → Get data → New data pipeline
2) Copy Data → Use copy assistant → Choose data source (Copy Data wizard) → Choose HTTP (New Resource)
    - URL: https://raw.githubusercontent.com/MicrosoftLearning/dp-data/main/sales.csv
    - Connection: Create new connection
    - Connection name: Specify a unique name
    - Data gateway: (none)
    - Authentication kind: Anonymous
3) Connect Data Source → (Enter appropriate data) → Next
4) Connected to Data Destination → (Enter appropriate data) → Next → Save + Run

## Create a notebook ##
1) Home → Open notebook → New notebook

In [ ]:
# Toggle parameter cell
table_name = "sales"

In [ ]:
from pyspark.sql.functions import *

# Read the new sales data
df = spark.read.format("csv").option("header","true").load("Files/new_data/*.csv")

## Add month and year columns
df = df.withColumn("Year", year(col("OrderDate"))).withColumn("Month", month(col("OrderDate")))

# Derive FirstName and LastName columns
df = df.withColumn("FirstName", split(col("CustomerName"), " ").getItem(0)).withColumn("LastName", split(col("CustomerName"), " ").getItem(1))

# Filter and reorder columns
df = df["SalesOrderNumber", "SalesOrderLineNumber", "OrderDate", "Year", "Month", "FirstName", "LastName", "EmailAddress", "Item", "Quantity", "UnitPrice", "TaxAmount"]
# Load the data into a table
df.write.format("delta").mode("append").saveAsTable(table_name)